# Jeff Pinegar
Project 2: ETL 
Due Dec. 23, 2022

### Extract and Transform European IPC Sales Data
---

In [1]:
# Import needed libraries
import pandas as pd     # if you get an error message module not found you need to add this to your environment.
import os               # this tells your project about the OS of your machine.  This will let you create machine independent notebooks
import numpy as np      # loads in the math tools
from datetime import datetime

---
### Extract

In [28]:
# Import data file
# My Jupyter notebook is in the current directory " . "
# There is a sub directory "Resources" and the file "IPC shipped from Germany by serial number 2022 (until 5.12.2022).csv" is located there.
sales_data = os.path.join('.','Resources', 'IPC serial number 2022 (until 5.12.2022).csv')            

# Opens the CVS file and read it in as a panda dataframe then print the head.
dfs_raw = pd.read_csv(sales_data, encoding="utf-8")                    
dfs_raw.head()

,Sales document number,Sales position,Delivery number,Delivery position,Serial number,Material,Product key,Net price,Currency,Country key,Created on,Possibly not the right Serial number
0,1.314044e+09,1000.0,8.220032e+09,10.0,23198864,1047323-00,DRBAAB,246.84,EUR,CZ,1/3/2022,0.0
1,1.314044e+09,2000.0,8.220032e+09,20.0,8220032281000020,1047367-00,DRBAAZ,52.27,EUR,CZ,1/3/2022,1.0
2,1.314044e+09,3000.0,8.220032e+09,30.0,8220032281000030,1047373-00,DRBABA,261.36,EUR,CZ,1/3/2022,1.0
3,1.314177e+09,3000.0,8.220031e+09,110.0,202111035073,2400456-06,DRBABB,905.85,EUR,IT,1/3/2022,0.0
4,1.314088e+09,2000.0,8.220031e+09,10.0,202112145175,1148693-01,DRCABB,574.98,EUR,IT,1/3/2022,0.0


---
### Transform

In [36]:
# Drop rows with missing data
dfs = dfs_raw.dropna(how='any')

# Eliminate HMI's - keeping only IPC
      # The 'Product Key for IPC start with the 'DRB'
      # REFERENCE: df[df['A'].str.contains("hello", case=False)]

dfs = dfs[dfs['Product key'].str.contains("DRB", case = True)]
dfs.head()

,Sales document number,Sales position,Delivery number,Delivery position,Serial number,Material,Product key,Net price,Currency,Country key,Created on,Possibly not the right Serial number
0,1.314044e+09,1000.0,8.220032e+09,10.0,23198864,1047323-00,DRBAAB,246.84,EUR,CZ,1/3/2022,0.0
1,1.314044e+09,2000.0,8.220032e+09,20.0,8220032281000020,1047367-00,DRBAAZ,52.27,EUR,CZ,1/3/2022,1.0
2,1.314044e+09,3000.0,8.220032e+09,30.0,8220032281000030,1047373-00,DRBABA,261.36,EUR,CZ,1/3/2022,1.0
3,1.314177e+09,3000.0,8.220031e+09,110.0,202111035073,2400456-06,DRBABB,905.85,EUR,IT,1/3/2022,0.0
9,1.314101e+09,3000.0,8.220033e+09,10.0,CI3521648399,1050387-09,DRBAAB,121.00,EUR,CZ,1/3/2022,0.0


In [37]:
# Create a function to check the status of a data frame.  The argument is the name of a dataframe.
def df_status(df_to_check):
      print(f'------------ Data type: ------------ \n {df_to_check.dtypes}' )
      print(f'\n \n ------------ Shape:  ------------ \n {df_to_check.shape}' )
      print(f'\n \n ------------Columns: ------------ \n {df_to_check.columns} \n')
      print (df_to_check.head(2))

df_status(dfs)

In [39]:
# remove entries with defective or missing serial numbers
# The data set contains a field indicating defective serial numbers 1=defective serial number
# REFERENCE:  a = oo_df.loc[oo_df['Medal']=='Gold']

dfs = dfs.loc[dfs['Possibly not the right Serial number']==0]
dfs.head()

,Sales document number,Sales position,Delivery number,Delivery position,Serial number,Material,Product key,Net price,Currency,Country key,Created on,Possibly not the right Serial number
0,1.314044e+09,1000.0,8.220032e+09,10.0,23198864,1047323-00,DRBAAB,246.84,EUR,CZ,1/3/2022,0.0
3,1.314177e+09,3000.0,8.220031e+09,110.0,202111035073,2400456-06,DRBABB,905.85,EUR,IT,1/3/2022,0.0
9,1.314101e+09,3000.0,8.220033e+09,10.0,CI3521648399,1050387-09,DRBAAB,121.00,EUR,CZ,1/3/2022,0.0
10,1.314101e+09,3000.0,8.220033e+09,10.0,CI3521648400,1050387-09,DRBAAB,121.00,EUR,CZ,1/3/2022,0.0
11,1.313988e+09,3000.0,8.220033e+09,10.0,202112175063,2404519,DRBAAB,720.00,EUR,IT,1/3/2022,0.0


In [40]:
# reduce the data frame to the needed columns and update the column names
dfs = dfs[[ 'Serial number'  ,  'Created on'  ,  'Net price'  ,  'Country key', 'Currency' ]]

# oo_df = oo_df.rename(columns={"Event_gender":"Event Gender"})
dfs = dfs.rename(columns={ 'Serial number' : 'Serial_No'  ,  'Created on' : 'Sell_Date'  ,  'Net price' : 'Sell_Price'  ,  'Country key' : 'Country' })

# Remove any duplicate orders
dfs = dfs.drop_duplicates(keep='first')

dfs.head()

,Serial_No,Sell_Date,Sell_Price,Country,Currency
0,23198864,1/3/2022,246.84,CZ,EUR
3,202111035073,1/3/2022,905.85,IT,EUR
9,CI3521648399,1/3/2022,121.00,CZ,EUR
10,CI3521648400,1/3/2022,121.00,CZ,EUR
11,202112175063,1/3/2022,720.00,IT,EUR


#### Convert Data types

In [42]:
# Convert "Sell_Price" to a float

# Remove commas from 'Net_Price'
dfs['Sell_Price'] = dfs['Sell_Price'].str.replace(r',','')

# Convert column "Sell_Price" from a string to a numeric 
dfs['Sell_Price'] = pd.to_numeric(dfs['Sell_Price']) 

In [43]:
# Convert sales date to datetime

# Define a funtion to change the format
def convertdate(date_in_dmyyyy_format):
    date_as_string= str(date_in_dmyyyy_format)
    newformat = datetime.strptime(date_as_string, "%m/%d/%Y").strftime("%Y-%m-%d")
    return str(newformat)

#reformat the sell_date string to yyyy-mm-dd.
dfs['Sell_Date']=dfs['Sell_Date'].apply(convertdate) 

# Convert the date string to type datetime
dfs['Sell_Date']=pd.to_datetime(dfs['Sell_Date'])


In [45]:
# which currencies do we have in the data set
Currencies = list(dfs['Currency'].unique())
Currencies

['EUR', 'USD', 'NOK', 'GBP', 'JPY']

In [13]:
# Convert the sales in local currency to Euros
# REFERENCE: https://medium.com/analytics-vidhya/convert-currencies-automatically-with-python-python-in-audit-2-6c574dbae44
from forex_python.converter import CurrencyRates
from datetime import date

c=CurrencyRates()
dfs_Euro = dfs

# lambda function lambda x: c.convert( x.Currency, 'EUR', x.Sell_Price, x.Sell_Date ), axis = 1
            # x               x represents each row
            # EUR             This is the currency that the sale is being converted to
            # x.Sell_price    This is the sell price in the local currency
            # x.Sell_Date     This is the date of the transaction. The rate on that date is for the conversion.
            # axis = 1        I don't fully understand the reason for this but it is needed to run through all the rows.

# with 8,400 row this takes about 9 minutes to run.
dfs_Euro['Sale_Price_EUR']=dfs_Euro.apply(lambda x: c.convert( x.Currency, 'EUR', x.Sell_Price, x.Sell_Date ), axis = 1)

# Presenver the dfs_Euro data frame so that it does not have to be generated again if there is an error later because it take 9 minutes
dfs = dfs_Euro  

In [49]:
# remove Sell_Price
del dfs['Sell_Price']

# save the clean data frame
df_clean = dfs

## Save the clean data as a CSV file

In [ ]:
# Export as CSV file
file_one = os.path.join('.','Resources', 'Sales_Data_Clean.csv')
dfs_clean.to_csv(file_one, index=False, header=True)                    # Writes out MyFileName.cvs into the subdirectory "Output"

In [ ]:
df_status(dfs_clean)

In [ ]:
df_status(dfs_Euro)